In [1]:
from tensorflow.keras import Model, layers

enc_input = layers.Input(shape = (4,171))
_, state_h, state_c = layers.LSTM(128,return_state = True)(enc_input)

dec_input = layers.Input(shape = (3,171))
lstm_out = layers.LSTM(128,return_sequences = True)(dec_input,initial_state =[state_h,state_c])

dec_output = layers.Dense(171,activation='softmax')(lstm_out)

model = Model(inputs = [enc_input,dec_input],outputs = dec_output)
model.summary()

2025-05-29 20:58:51.554787: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-05-29 20:58:51.554802: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-05-29 20:58:51.554808: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-05-29 20:58:51.554821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-29 20:58:51.554832: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │    153,600 │ input_layer[0][0] │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 3, 128)    │    153,600 │ input_layer_1[0]… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 171)    │     22,059 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 329,259 (1.26 MB)

 Trainable params: 329,259 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
model.compile(loss = 'sparse_categorical_crossentropy',optimizer='adam')

In [3]:
import pandas as pd
import numpy as np

arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('data/korean.csv',header = None)

num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))
char_to_num = {char:i for i, char in enumerate(num_to_char)}

171


In [4]:
raw = pd.read_csv('data/translate.csv',header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [5]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
temp_kor = '사랑'
np.eye(171)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 171)

In [6]:
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = []
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])

        kor = [char_to_num[c] for c in ('S'+ seq[1])]
        dec_in.append(np.eye(171)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out,axis=2)
    
    return enc_in,dec_in,rnn_out


In [7]:
X_enc,X_dec,y_rnn = encode(eng_kor)

In [8]:
model.fit([X_enc,X_dec],y_rnn,epochs = 500)

Epoch 1/500


2025-05-29 20:58:52.046243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 5.1357
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.1061
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.0737
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.0328
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.9754
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.8924
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.7699
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 4.5655
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.2119
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.6627
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.3404
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.3834
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.3211
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.2529
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.2561
Epoch 16/500
4/4 ━━━━━━━━━━━━━━

In [9]:
enc_in,dec_in, _ = encode([['tall','PP']])
pred = model.predict([enc_in,dec_in])

word = np.argmax(pred[0],axis = 1)

num_to_char[word[0]],num_to_char[word[1]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


('시', '다')

In [10]:
from numpy.random import randint

pick = randint(0,len(eng_kor),5)
choose = [[eng_kor[i][0],'PP'] for i in pick]
print(choose)

[['dust', 'PP'], ['root', 'PP'], ['weak', 'PP'], ['wind', 'PP'], ['hole', 'PP']]


In [11]:
enc_in,dec_in,_ = encode(choose)
pred = model.predict([enc_in,dec_in])
print(pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
(5, 3, 171)


In [12]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i],axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng,kor)

dust 시선
root 사다
weak 소다
wind 바선
hole 사다


In [ ]:
#어텐션 이용
#Seq2Seq는 하나의 벡터로 압축하는 과정에서 입력 시퀀스의 일부 정보가 손실됌.
#어텐션 함수는 Q(Query), K(Keys), V(Values)를 매개변수로 사용
#Q(Query)는 특정 시점에서의 디코더 셀의 은닉 상태
#K(Key)는 모든 시점에서의 인코더 셀의 Q를 반영하기 전 은닉 상태
#V(Value)는 모든 시점에서의 인코더 셀의 Q 반영 후 은닉 상태

#① 어텐션 가중치(Attention Weights) 계산
#② 어텐션 분포(Attention Distribution) 계산
#③ 어텐션 벡터(Attention Vector) 계산
#④ 연결(Concatenation)

import tensorflow as tf
from tensorflow.keras import Model, layers
# 인코더 입력 정의: 영문 단어가 입력, 각 단어는 4자 길이, 모든 문자의 수는 171개
enc_input = layers. Input (shape=(4, 171))
# 인코더 LSTM 정의: 모든 타임스템의 출력을 반환하도록 설정
enc_output, state_h, state_c = layers.LSTM(128, return_sequences = True, return_state=True)(enc_input)
# 디코더 입력 정의
dec_Input= layers. Input (shape=(3, 171)) #한글 단어는 2자 길이 +<Start> 토큰, 모든 문자의 수는 171개
# 디코더 LSTM 정의: 모든 시킨스의 출력을 반환하도록 설정
dec_lstm_output,_,_ = layers.LSTM(128, return_sequences=True, return_state=True) (dec_Input, initial_state= [state_h, state_c])
# 어텐션 메커니즘 정의
context_vector = layers.Attention() ([dec_lstm_output,enc_output])
# 컨텍스트 벡터와 디코더 LST 출력을 결합
context_and_lstm_output = layers.Concatenate() ([context_vector, dec_lstm_output ])
# 디코더 출력층 정의: 출력 크기는 모든 문자의 수인 171, oftmax 활성화 함수를 사용
output = layers.Dense(171, activation="softmax")(context_and_lstm_output)
# 모델 정의: 인코더 입력(enc_Input)과 디코더 입력(dec_Input)을 모델의 입력으로, 디코더 출력을 모델
model = Model(inputs=[enc_input, dec_input], outputs=[output])
# 모델 요약 출력
model. summary ()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 4, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, 3, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ [(None, 4, 128),  │    153,600 │ input_layer_20[0… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_20 (LSTM)      │ [(None, 3, 128),  │    153,600 │ input_layer_21[0… │
│                     │ (None, 128),      │            │ lstm_19[0][1],    │
│                     │ (None, 128)]      │            │ lstm_19[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_6         │ (None, 3, 128)    │          0 │ lstm_20[0][0],    │
│ (Attention)         │                   │            │ lstm_19[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 3, 256)    │          0 │ attention_6[0][0… │
│ (Concatenate)       │                   │            │ lstm_20[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 3, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3, 171)    │     43,947 │ concatenate_6[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 351,147 (1.34 MB)

 Trainable params: 351,147 (1.34 MB)

 Non-trainable params: 0 (0.00 B)